## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-25-16-55-59 +0000,bbc,220 MPs call for Starmer to recognise Palestin...,https://www.bbc.com/news/articles/cx202zvygmlo
1,2025-07-25-16-55-28 +0000,nypost,Trump OKs Israel annihilating Hamas: ‘I think...,https://nypost.com/2025/07/25/us-news/trump-ok...
2,2025-07-25-16-53-19 +0000,nyt,"Justice Dept. Officials File Suit, Saying They...",https://www.nytimes.com/2025/07/25/us/politics...
3,2025-07-25-16-47-20 +0000,nypost,AOC broke House rules to attend Met Gala with ...,https://nypost.com/2025/07/25/us-news/aoc-brok...
4,2025-07-25-16-44-40 +0000,nypost,"Venmo, PayPal users can now send money to the ...",https://nypost.com/2025/07/25/business/venmo-p...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2376/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,59
178,gaza,16
196,new,14
368,epstein,13
6,israel,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
200,2025-07-25-01-00-00 +0000,wsj,Streetwise: The Trump administration has rampe...,https://www.wsj.com/economy/central-banking/tr...,142
44,2025-07-25-15-18-56 +0000,bbc,Trump says Hamas 'didn't want deal' as US and ...,https://www.bbc.com/news/articles/crrqrz857wgo,123
210,2025-07-24-23-54-32 +0000,nyt,Powell Tells Trump He Was Wrong About Renovati...,https://www.nytimes.com/2025/07/24/us/politics...,115
190,2025-07-25-02-10-00 +0000,wsj,Republican National Committee Chair Michael Wh...,https://www.wsj.com/politics/elections/north-c...,114
201,2025-07-25-01-00-00 +0000,wsj,Buoyed by voters’ improving views of the econo...,https://www.wsj.com/politics/policy/trump-appr...,113


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
200,142,2025-07-25-01-00-00 +0000,wsj,Streetwise: The Trump administration has rampe...,https://www.wsj.com/economy/central-banking/tr...
284,81,2025-07-24-19-57-36 +0000,cbc,"Israel, U.S. pull out of Gaza ceasefire talks ...",https://www.cbc.ca/news/world/france-macron-pa...
201,54,2025-07-25-01-00-00 +0000,wsj,Buoyed by voters’ improving views of the econo...,https://www.wsj.com/politics/policy/trump-appr...
214,54,2025-07-24-23-50-13 +0000,nypost,Former top Biden White House official blames M...,https://nypost.com/2025/07/24/us-news/former-t...
51,48,2025-07-25-14-57-12 +0000,nypost,Tens of thousands flee their homes in Thailand...,https://nypost.com/2025/07/25/world-news/tens-...
112,45,2025-07-25-11-05-54 +0000,nypost,Cat mom accuses Blue Angels of terrorizing her...,https://nypost.com/2025/07/25/us-news/cat-mom-...
109,41,2025-07-25-11-28-41 +0000,nypost,Sickos accused of running child sex-traffickin...,https://nypost.com/2025/07/25/us-news/sickos-a...
17,40,2025-07-25-16-25-00 +0000,wsj,President Trump has long wanted a U.S. soverei...,https://www.wsj.com/politics/policy/sovereign-...
79,40,2025-07-25-13-46-47 +0000,nypost,Arizona woman who admitted to helping North Ko...,https://nypost.com/2025/07/25/us-news/arizona-...
10,38,2025-07-25-16-36-09 +0000,nypost,Trump forecasts ‘50-50 chance’ for EU trade de...,https://nypost.com/2025/07/25/us-news/trump-fo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
